In [15]:
import polars  as pl
from perspective.widget import PerspectiveWidget

In [24]:
df = pl.read_parquet("stock_trades.parquet")
df = df.sort("交易日期", "交易时间", "证券代码")
df = df.with_columns(
    pl.col("交易时间").cast(pl.String),
    手续费率= pl.col("手续费")/pl.col("成交金额"),
    印花税率= pl.col("印花税")/pl.col("成交金额"),
    过户费率= pl.col("过户费")/pl.col("成交金额"),
)
df = df.with_row_index("序号",1)
df

序号,券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率
u32,str,date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""湘财""",2022-07-11,"""09:33:37""","""000900""","""现代投资""","""买入""",4.05,34400.0,139320.0,22.29,0.0,1.39,0.0,-139342.29,0.00016,0.0,0.00001
2,"""湘财""",2022-07-11,"""09:34:24""","""601077""","""渝农商行""","""买入""",3.65,38300.0,139795.0,22.37,0.0,1.38,0.0,-139818.75,0.00016,0.0,0.00001
3,"""湘财""",2022-07-11,"""09:36:30""","""600894""","""广日股份""","""买入""",6.54,21400.0,139956.0,22.39,0.0,1.41,0.0,-139979.8,0.00016,0.0,0.00001
4,"""湘财""",2022-07-11,"""09:37:25""","""601992""","""金隅集团""","""买入""",2.59,54000.0,139860.0,22.38,0.0,1.42,0.0,-139883.8,0.00016,0.0,0.00001
5,"""湘财""",2022-07-11,"""09:38:16""","""002462""","""嘉事堂""","""买入""",13.51,10400.0,140504.0,22.48,0.0,1.41,0.0,-140526.48,0.00016,0.0,0.00001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
359,"""海通两融""",2023-10-31,"""09:31:53""","""002956""","""西麦食品""","""卖出""",14.13,5000.0,70650.0,6.74,35.35,0.0,0.0,70607.91,0.000095,0.0005,0.0
360,"""海通两融""",2023-10-31,"""09:39:57""","""603214""","""爱婴室""","""买入""",15.84,3100.0,49104.0,5.0,0.0,0.51,0.0,-49109.51,0.000102,0.0,0.00001
361,"""海通两融""",2023-10-31,"""09:40:55""","""300132""","""青松股份""","""买入""",5.21,9600.0,50016.0,5.0,0.0,0.0,0.0,-50021.0,0.0001,0.0,0.0


In [25]:
PerspectiveWidget(df)

PerspectiveWidget(binding_mode='server', columns=['序号', '券商', '交易日期', '交易时间', '证券代码', '证券名称', '买卖标志', '成交价格', …

In [45]:
d1= df.join(
    df.group_by("证券代码","证券名称")
    .agg(
        结余数量 = 
            pl.when(pl.col("买卖标志") == "卖出")
            .then(-pl.col("成交数量"))
            .when(pl.col("买卖标志") == "买入")
            .then(pl.col("成交数量"))
            .sum()
    )
    .filter(pl.col("结余数量") < 0 ),
    on ="证券代码",
    how ="anti",
)

In [48]:
d1

序号,券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,手续费率,印花税率,过户费率
u32,str,date,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""湘财""",2022-07-11,"""09:33:37""","""000900""","""现代投资""","""买入""",4.05,34400.0,139320.0,22.29,0.0,1.39,0.0,-139342.29,0.00016,0.0,0.00001
2,"""湘财""",2022-07-11,"""09:34:24""","""601077""","""渝农商行""","""买入""",3.65,38300.0,139795.0,22.37,0.0,1.38,0.0,-139818.75,0.00016,0.0,0.00001
3,"""湘财""",2022-07-11,"""09:36:30""","""600894""","""广日股份""","""买入""",6.54,21400.0,139956.0,22.39,0.0,1.41,0.0,-139979.8,0.00016,0.0,0.00001
4,"""湘财""",2022-07-11,"""09:37:25""","""601992""","""金隅集团""","""买入""",2.59,54000.0,139860.0,22.38,0.0,1.42,0.0,-139883.8,0.00016,0.0,0.00001
5,"""湘财""",2022-07-11,"""09:38:16""","""002462""","""嘉事堂""","""买入""",13.51,10400.0,140504.0,22.48,0.0,1.41,0.0,-140526.48,0.00016,0.0,0.00001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
359,"""海通两融""",2023-10-31,"""09:31:53""","""002956""","""西麦食品""","""卖出""",14.13,5000.0,70650.0,6.74,35.35,0.0,0.0,70607.91,0.000095,0.0005,0.0
360,"""海通两融""",2023-10-31,"""09:39:57""","""603214""","""爱婴室""","""买入""",15.84,3100.0,49104.0,5.0,0.0,0.51,0.0,-49109.51,0.000102,0.0,0.00001
361,"""海通两融""",2023-10-31,"""09:40:55""","""300132""","""青松股份""","""买入""",5.21,9600.0,50016.0,5.0,0.0,0.0,0.0,-50021.0,0.0001,0.0,0.0


In [50]:
start_date= df["交易日期"].min()
start_date

datetime.date(2022, 7, 11)

In [51]:
end_date= df["交易日期"].max()
end_date

datetime.date(2023, 10, 31)

In [54]:
k1= pl.select(
    日期= pl.date_range(start_date, end_date)
)
k1

日期
date
2022-07-11
2022-07-12
2022-07-13
2022-07-14
2022-07-15
…
2023-10-27
2023-10-28
2023-10-29


In [58]:
k2= df["证券代码"].unique().sort().to_frame()
k2

证券代码
str
"""000096"""
"""000532"""
"""000559"""
"""000599"""
"""000655"""
…
"""688299"""
"""688321"""
"""688360"""


In [62]:
k= k1.join(k2, how= "cross")

In [72]:
d2= k.join(d1,left_on= ["日期","证券代码"], right_on= ["交易日期","证券代码"],how= "left").sort("日期","证券代码").with_columns(
    结余数量 = 
            pl.when(pl.col("买卖标志") == "卖出")
            .then(-pl.col("成交数量"))
            .when(pl.col("买卖标志") == "买入")
            .then(pl.col("成交数量"))
            .otherwise(0).cum_sum().over("证券代码")
).filter(pl.col.结余数量 > 0)

In [73]:
PerspectiveWidget(d2)

PerspectiveWidget(binding_mode='server', columns=['日期', '证券代码', '序号', '券商', '交易时间', '证券名称', '买卖标志', '成交价格', '成…